In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

from time import sleep
import json

In [6]:
# pip install selenium==4.9.1

In [7]:
def inicializaDriver():
    # Necessario para executar no colab
    # !pip install selenium
    # !wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
    # !sudo apt install ./google-chrome-stable_current_amd64.deb

    chrome_options = webdriver.ChromeOptions()
    # chrome_options.add_argument('--headless') # Necessario para executar no colab
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(chrome_options=chrome_options)
    
    return driver

def pesquisaPorBairro(driver, bairro):
    # Vai para a página inicial
    url = 'https://www.zapimoveis.com.br/'
    driver.get(url)
    sleep(3)
    
    driver.find_element(By.XPATH, "//input[@placeholder='Digite o nome da rua, bairro ou cidade']").send_keys(bairro)
    sleep(3)
    
    select_list_bairro = driver.find_elements(By.CLASS_NAME, "l-multiselect__input")
    bairro_correto = select_list_bairro[0]
    bairro_correto.click()
    sleep(1)
    
    driver.find_element(By.XPATH, "//*[text()='Buscar']").click()
    sleep(3)
    
def coletaListaDeImoveisDaPagina(driver):
    lista_de_imoveis = driver.find_elements(By.CSS_SELECTOR, 'a.result-card') # Procure elementos com a tag <a> e classe "result-card"

    lista_imoveis_json = []

    for imovel in lista_de_imoveis:
        dados_imovel = imovel.find_element(By.CLASS_NAME, "l-card__content")
        
        rua = dados_imovel.find_element(By.CLASS_NAME, "card__location").find_element(By.CSS_SELECTOR, 'p').text
        preco = dados_imovel.find_element(By.CLASS_NAME, "listing-price").text
        area = dados_imovel.find_element(By.XPATH, ".//p[@itemprop='floorSize']").text
        descricao = dados_imovel.find_elements(By.CSS_SELECTOR, "p.l-text")[1].text
        
        imovel_json = {"rua": rua,
                    "preco": preco,
                    "area": area,
                    "descricao": descricao}
        
        lista_imoveis_json.append(imovel_json)
        
    return lista_imoveis_json

def escreveArquivoDadosCompilados(lista_imoveis_json):
    # Criando o arquivo json com os dados obtidos

    filename = 'imoveis.json'
    dados_json = json.dumps(lista_imoveis_json, ensure_ascii=False).encode("utf-8")
    with open(filename, "w", encoding='utf8') as file:
        json.dump(lista_imoveis_json, file, ensure_ascii=False)
    sleep(1)
    
def scrollAteFinalPagina(driver):
    for i in range(0,100):
        body = driver.find_element(By.CSS_SELECTOR, 'body')
        body.send_keys(Keys.PAGE_DOWN)
        sleep(0.5)

In [8]:
driver = inicializaDriver()

bairro = 'Pinheiros, São Paulo - SP'

pesquisaPorBairro(driver, bairro)

lista_imoveis_json = [] # irá guardar os dados dos imoveis de todas as páginas

for page in range(1,4): # Páginas 1, 2 e 3
    scrollAteFinalPagina(driver)
    
    lista_imoveis_pagina = coletaListaDeImoveisDaPagina(driver)
    lista_imoveis_json.extend(lista_imoveis_pagina)
    
    if(page == 1):
        urlSemPagina = driver.current_url[:-1] # [:-1] remove o ultimo caractere da url
        
    urlProximaPagina = urlSemPagina+str(page+1) # ...&pagina=3
    driver.get(urlSemPagina+str(page+1)) # Acessa a próxima página

driver.quit()


C:\Users\woodg\AppData\Local\Temp\ipykernel_19512\2854766598.py:11: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=chrome_options)


In [9]:
!pip uninstall selenium 
!pip install selenium==4.9.1


^C


In [ ]:
lista_imoveis_json


In [ ]:
# Tratar os dados
import re

# Limpando os imoveis que não possuem preço ou m²
for imovel in lista_imoveis_json:
    if imovel['preco'] == "" or imovel['area'] == "":
        index_to_delete = lista_imoveis_json.index(imovel)
        del lista_imoveis_json[index_to_delete]

# Deixando somente inteiros nos campos "area" e "preco"
inteiro_incluindo_pontos = r'[\d,.]+'

for imovel in lista_imoveis_json:
    
    index = lista_imoveis_json.index(imovel)
    
    # campo "area"
    lista_imoveis_json[index]['area'] = re.search(r'\d+', imovel['area']).group() # Pega só o primeiro número, sem pontos, sem vírgulas e sem espaços
    
    # campo "preco"
    try:
        lista_imoveis_json[index]['preco'] = re.findall(inteiro_incluindo_pontos, imovel['preco'])[0]
    except:
        del lista_imoveis_json[index]
    
    # removendo pontos do "preco"
    lista_imoveis_json[index]['preco'] = lista_imoveis_json[index]['preco'].replace(".", "")
    
print(lista_imoveis_json)

escreveArquivoDadosCompilados(lista_imoveis_json)